In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pathlib
import cv2
import pathlib
import torch
import torchvision.transforms as transforms
from torchvision.io import read_image

DATA_DIR = "facial_expressions/data"
IMAGE_DIR = "facial_expressions/images"

legend = pd.read_csv(f'{DATA_DIR}/legend.csv')
legend['emotion'] = legend['emotion'].str.lower()
image_paths = list(pathlib.Path(IMAGE_DIR).glob('*.jpg'))
images = [plt.imread(str(path)).astype('float32') for path in image_paths]

In [2]:
data = pd.DataFrame({"path": [p.name for p in image_paths], "image": images})
data = data.join(legend.set_index('image'), on='path')
data.shape

(13726, 4)

In [3]:
data['shape'] = [img.shape for img in data['image']]

In [4]:
same_shape_data = data[data['shape'] == (350, 350)]
same_shape_data.shape

(12801, 5)

In [5]:
# tensor_img = torch.Tensor(data['image'])
# tensor_label = torch.Tensor(data['emotion'])
# convert_tensor = transforms.ToTensor()
# convert_tensors = np.vectorize(convert_tensor)
# ts = [convert_tensor(img) for img in images]
# tensor_big = torch.Tensor(ts)

In [6]:
from torch.utils.data import Dataset
import pandas as pd
import os
from torchvision.io import read_image

class FacialExpression(Dataset):
  def __init__(self, annotations_file=f'{DATA_DIR}/legend.csv', img_dir=IMAGE_DIR, transform=None, target_transform=None):
    self.img_dir = img_dir
    self.data = same_shape_data
    self.classes = list(self.data['emotion'].unique())
    self.data['class'] = [self.classes.index(e) for e in self.data['emotion']]
    self.img_labels = self.data['class']
    self.transform = transform
    self.target_transform = target_transform

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self, idx):
    image = self.data['image'][idx]
    label = self.img_labels[idx]
    if self.transform:
        image = self.transform(image)
    if self.target_transform:
        label = self.target_transform(label)
    return image, label

In [ ]:
trainset = FacialExpression()

In [26]:
batch_size = 4
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [27]:
# import matplotlib.pyplot as plt
# import numpy as np
# import torchvision

# # functions to show an image


# def imshow(img):
#     # img = img / 2 + 0.5     # unnormalize
#     # npimg = img.numpy()
#     # plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.imshow(img, cmap="gray")
#     plt.show()


# # get some random training images
# dataiter = iter(trainloader)
# images, labels = next(dataiter)

# # show images
# imshow(torchvision.utils.make_grid(images))
# # print labels
# print(' '.join(f'{labels[j]:5s}' for j in range(batch_size)))

In [28]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [31]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [32]:
for epoch in range(2):  # loop over the dataset multiple times

  running_loss = 0.0
  for i, data in enumerate(trainloader, 0):
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data
    print(inputs.shape)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    running_loss += loss.item()
    if i % 2000 == 1999:    # print every 2000 mini-batches
      print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
      running_loss = 0.0

print('Finished Training')

torch.Size([4, 350, 350])


RuntimeError: Given groups=1, weight of size [6, 3, 5, 5], expected input[1, 4, 350, 350] to have 3 channels, but got 4 channels instead

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cpu
